In [2]:
using Pkg
Pkg.activate("../../env/.")
Pkg.instantiate()

  Activating project at `~/analogous_neural/env`


In [3]:
include("../../src/analogous_bars.jl")
include("../../src/Eirene_var.jl")

using .analogous_bars
using .Eirene_var


WebIO._IJuliaInit()

In [27]:
using DelimitedFiles
#using HDF5
using Plots
#using FileIO
#using MultivariateStats
#using CSV
#using DataFrames
using Distances

# 1. Load data 

In [39]:
# load coordinates of point clouds
PC1 = readdlm("pointcloud1.csv", ',', Float64)
PC2 = readdlm("pointcloud2.csv", ',', Float64);

n_PC1 = size(PC1, 1)
n_PC2 = size(PC2, 1)

println("number of points in PC1: ", n_PC1)
println("number of points in PC2: ", n_PC2)

number of points in PC1: 625
number of points in PC2: 1290


In [40]:
# Compute dissimilarity matrices 

PC_total = vcat(PC1, PC2)
D = pairwise(Euclidean(), PC_total, dims=1)

D1 = D[1:n_PC1, 1:n_PC1]
D2 = D[n_PC1+1:end, n_PC1+1:end]
D_cross = D[1:n_PC1, n_PC1+1:end]

println("Shape of D: ", size(D))
println("Shape of D1: ", size(D1))
println("Shape of D2: ", size(D2))
println("Shape of D_cross: ", size(D_cross))

Shape of D: (1915, 1915)
Shape of D1: (625, 625)
Shape of D2: (1290, 1290)
Shape of D_cross: (625, 1290)


# 2. Run persistence

In [41]:
C1 = eirene(D1, record = "all", maxdim = 1)
C2 = eirene(D2, record = "all", maxdim = 1)
W = compute_Witness_persistence(D_cross)

barcode_C1 = barcode(C1, dim = 1)
barcode_C2 = barcode(C2, dim = 1)
barcode_W = barcode(W["eirene_output"], dim = 1);

LoadError: InterruptException:

In [ ]:
# plot all persistence diagrams & significance feature cutoff
selected_1, cutoff_1 = select_persistent_intervals_IQR(barcode_C1)
p1 = plot_PD(barcode_C1; 
            cutoff = cutoff_1,  
            title = "PD(pc1)", 
            titlefontsize = 12)

selected_W, cutoff_W = select_persistent_intervals_IQR(barcode_W)
p2 = plot_PD(barcode_W; 
            cutoff = cutoff_W, 
            title = "DPD", 
            titlefontsize = 12)

selected_2, cutoff_2 = select_persistent_intervals_IQR(barcode_C2)
p3 = plot_PD(barcode_C2; 
            cutoff = cutoff_2, 
            title = "PD(pc2)", 
            titlefontsize = 12)

plot(p1, p2, p3, layout = grid(1,3))